Übung: Dedizierte Docker Image-Stände bauen und verwalten
----------------------------------------------------------------------------------------

Nun wollen wir ein Docker Image erstellen, dazu brauchen wir ein Dockerfile.

Im Verzeichnis ist schon eines Vorhanden und wir können es einfach mal Ausgeben:

In [ ]:
! wget https://raw.githubusercontent.com/mc-b/devops/master/docker/mysql/Dockerfile -O Dockerfile
! cat Dockerfile

Um das Anschliessend das Docker Image zu bilden

In [ ]:
! docker build -t mysql .

Das Ergebnis können wir uns mit `docker image ls` anschauen

In [ ]:
! docker image ls

Ebenfalls können die einzelnen Buildschritte sichtbar gemacht werden

In [ ]:
! docker history mysql